# **Checkpoint 5: Natural Language Processing**

# **Installations**

In [12]:
!pip install bertopic
!pip install bertopic[visualization]

# **Downloading Files (Required)**

In [13]:
!wget https://raw.githubusercontent.com/dmavrofridis/The-Exalted-Spartans/main/checkpoint-5/src/allegation_race_gender_loc_with_text.sql

--2021-12-01 05:29:07--  https://raw.githubusercontent.com/dmavrofridis/The-Exalted-Spartans/main/checkpoint-5/src/allegation_race_gender_loc_with_text.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889 [text/plain]
Saving to: ‘allegation_race_gender_loc_with_text.sql.2’

allegation_race_gen 100%[===================>]     889  --.-KB/s    in 0s      

2021-12-01 05:29:07 (43.7 MB/s) - ‘allegation_race_gender_loc_with_text.sql.2’ saved [889/889]



# **Imports**

In [14]:
# Main File for NLP
import pandas as pd
import numpy as np
import psycopg2
import bertopic
from copy import deepcopy
from bertopic import BERTopic
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Global Variables**

In [15]:
conn_string = "host='codd04.research.northwestern.edu' port='5433' dbname='postgres' user='cpdbstudent' password='DataSci4AI'"
sql_file = "allegation_race_gender_loc_with_text.sql"
text_cols = ["summary", "cr_text"]
unique_races = ["Hispanic", "Asian/Pacific Islander", "White", "Native American/Alaskan Native", "Black"]
unique_communities = ["Rogers Park", "Washington Park", "Archer Heights", "Jefferson Park", "Belmont Cragin", "Avalon Park", "West Ridge", "Near South Side", "Avondale", "O'Hare", "Armour Square", "Grand Boulevard", "Montclare", "Gage Park", "McKinley Park", "Austin", "Forest Glen", "Kenwood", "North Center", "Logan Square", "Chicago Lawn", "Albany Park", "Lincoln Square", "West Pullman", "Lincoln Park", "New City", "Uptown", "Fuller Park", "Near North Side", "Near West Side", "Calumet Heights", "East Side", "Lower West Side", "North Lawndale", "Bridgeport", "Greater Grand Crossing", "West Englewood", "West Elsdon", "Brighton Park", "Morgan Park", "Portage Park", "Chatham", "Mount Greenwood", "Ashburn", "West Town", "Lake View", "Garfield Ridge", "Auburn Gresham", "South Chicago", "Hegewisch", "Riverdale", "Norwood Park", "Beverly", "Oakland", "Dunning", "Hermosa", "Hyde Park", "South Shore", "South Deering", "Edgewater", "West Lawn", "East Garfield Park", "Clearing", "South Lawndale", "Irving Park", "Woodlawn", "Loop", "Douglas", "Washington Heights", "Englewood", "Humboldt Park", "West Garfield Park", "North Park", "Burnside", "Roseland", "Pullman"]
custom_stop_words = ["initial", "intake", "finding", "alleged", "alleg", "allege", "allegation","allegations", "allegedly", "alleges","accusations","accuseds", "alleging", "accuse","accuses", "accusing","accused", "officer", "chicago", "parties","party", "report","complain", "complaint", "complained", "police", "citation", "violation", "confidential", "reported", "reporting", "complaining", "complainant", "compalinant", "compiainant", "complaianant", "complainants", "complainantvictim", "complaintant", "complaints", "complamnant","complanant", "reportin", "reportina","reports", "partys", "partyvictim", "partyvictims"]
custom_punctuation = ['‘', '“', "-","”", "-", ".", ",", ":", "(", ")", "/", "]", "[", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

# **Functions**

In [16]:
def topics_per_race(race, df):
    return df.loc[df['race'] == race]['topics'].value_counts()


def topics_per_community(community, df):
    return df.loc[df['community'] == community]['topics'].value_counts()


def connect_to_db():
    return psycopg2.connect(conn_string)


def query_to_string():
    # Open and read the file as a single buffer
    fd = open(sql_file, 'r')
    sqlFile = fd.read()
    fd.close()
    return sqlFile

def clean_df_strings(df, column):
  stop_words = stopwords.words('english')
  stop_words += custom_stop_words + custom_punctuation
  df[column] = df[column].apply(lambda x: ' '.join([word for word in x.split() if word.lower().isalnum()]))
  df[column] = df[column].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop_words)]))
  return df


def generate_topics(docs, model):
  # docs = list(filter("".__ne__, docs))
  # docs = list(filter(None.__ne__, docs))

  # create model
  if model is None:
      model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

  topics, probabilities = model.fit_transform(docs)
  info = model.get_topic_info()
  freq = model.get_topic_freq()
  print("\n")
  print(info)
  print("\n")
  for i in range(len(freq)):
    print(model.get_topic(i))

  model.visualize_topics(top_n_topics=5)
  model.visualize_barchart(top_n_topics=5)

  # Save the model
  model.save("model")

  return topics, probabilities, info
  

# **Main (Actual Code)**

In [17]:
connection = connect_to_db()
sql_query = query_to_string()
# First step is to import the required tables
df = pd.read_sql(sql_query, connection)
df.name = "cp5"
model = None
print(df.shape)

# Before loading the model, we make sure to remove the stopwords and clean the 
# strings
print("BEFORE -> ", df['cr_text'].tolist()[0])
df = clean_df_strings(df, "cr_text")
print("\nAFTER -> ", df['cr_text'].tolist()[0])

try:
  model = BERTopic.load("model")
  print("\nModel exists, loading it now!\n")
except:
  print("\nModel does not exist yet, train it now!\n")

topics, probabilities, info = generate_topics(df['cr_text'].tolist(), model)
df["topics"] = topics
df["topics"] = df["topics"].map(info.set_index("Topic")["Name"])

for race in unique_races:
  print("\nCount of topics for race -> ", race)
  topic_count = topics_per_race(race, df)
  print(topic_count)

for community in unique_communities:
  print("\nCount of topics for community -> ", community)
  topic_count = topics_per_community(community, df)
  print(topic_count)


(1944, 5)
BEFORE ->  Initial / Intake Allegation 1:  The reporting party, who did not witness the
incident, alleged that an unknown Hispanic
male officer in plainclothes struck the victim
on the head with a gun.
Initial / Intake Allegation 1: The reporting party, who did not witness the
incident, alleged that an unknown Hispanic
male officer in plainclothes struck the victim
on the head with a gun
Finding 1: NO AFFIDAVIT


AFTER ->  witness unknown Hispanic male plainclothes struck victim head witness unknown Hispanic male plainclothes struck victim head gun AFFIDAVIT

Model exists, loading it now!



Batches:   0%|          | 0/61 [00:00<?, ?it/s]

2021-12-01 05:29:13,089 - BERTopic - Transformed documents to Embeddings
2021-12-01 05:29:24,460 - BERTopic - Reduced dimensionality with UMAP
2021-12-01 05:29:24,682 - BERTopic - Clustered UMAP embeddings with HDBSCAN




    Topic  Count                                      Name
0       0   1677       0_officers_victim_affidavit_refused
1       1     77       1_inventory_phone_cellular_illinois
2       2     32     2_indebtedness_debt_compliance_notice
3       3     25    3_city_indebtedness_indebted_detention
4       4     25  4_taser_accidentally_discharged_deployed
5      -1     19        -1_library_lieutenant_wallet_spark
6       5     19   5_nomisconduct_wires_mount_departmental
7       6     17            6_amount_indebtedness_city_pay
8       7     16            7_weapon_secure_serial_firearm
9       8     13          8_indebtness_amount_city_amounts
10      9     13       9_obtain_log_notification_discharge
11     10     11     10_inventory_amount_shortage_sergeant


[('officers', 0.03252203233243763), ('victim', 0.02568163388792611), ('affidavit', 0.02345356673896525), ('refused', 0.019279155921187837), ('arrested', 0.01838694365572626), ('handcuffed', 0.01817360931085595), ('grabbed', 0.016